# Gregg (2020) Replication

This notebook replicates the main results from Gregg (2020) on factory productivity and incorporation in late Imperial Russia.

In [8]:
# === IMPORTS ===
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [9]:
# === LOAD DATA ===
df = pd.read_stata("AG_Corp_Prod_Database.dta")
print(f"Data loaded successfully! Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

Data loaded successfully! Shape: (40288, 65)
Columns: ['id', 'Form', 'PSZ', 'PSZ1900', 'FoundingYear', 'Province', 'Region', 'Industry', 'OntheSide', 'Age', 'TaxedActivity', 'YEAR', 'PSZLastYear', 'PSZ1908', 'SubindustryCode', 'STCAP', 'Revenue', 'TotalWorkers', 'TotalPower', 'GrandTotalWorkers', 'RevperWorker', 'PowerperWorker', 'RevperGrandWorker', 'PowerperGrandWorker', 'logRevperWorker', 'logPowerperWorker', 'logRevperGrandWorker', 'logPowerperGrandWorker', 'logRev', 'logWorkers', 'logPower', 'RegIndGroup', 'RegIndYearGroup', 'ProvIndGroup', 'ProvIndYearGroup', 'IndYearGroup', 'IndustryFactor', 'ProvinceFactor', 'YearFactor', 'AKTS', 'PAI', 'factory_id', 'FormNextYear', 'FormNextNextYear', 'FactoryisCorpin1894', 'FormNextYearin1894', 'FactoryisCorpin1900', 'FormNextYearin1900', 'FactoryisCorpin1908', 'NEWDEV', 'SHARES', 'STPRICE', 'BONDS', 'Silk', 'Flax', 'Animal', 'Wool', 'Cotton', 'MixedMaterials', 'Wood', 'Paper', 'MetalsandMachines', 'Foods', 'Chemical', 'Mineral']


In [10]:
# === TABLE 3 CHECK: Pooled OLS ===
df_check = df.dropna(subset=["logRevperWorker", "Form", "Industry", "Province", "YEAR"])
model_check = smf.ols(
    formula="logRevperWorker ~ Form + C(Industry) + C(Province) + C(YEAR)",
    data=df_check
).fit(cov_type="HC1")

print("\n=== Table 3 Coefficient on Form (pooled OLS) ===")
print(f"Coefficient: {model_check.params['Form']:.4f}")
print(f"Std Error: {model_check.bse['Form']:.4f}")


=== Table 3 Coefficient on Form (pooled OLS) ===
Coefficient: 0.4900
Std Error: 0.0195


In [11]:
# === STEP 1: Construct TFP as residuals (Stata-style: no fixed effects) ===

# Filter for valid observations
df_tfp = df.dropna(subset=["logRev", "logWorkers", "logPower", "id"]).copy()

# Ensure numeric data types
df_tfp["logRev"] = pd.to_numeric(df_tfp["logRev"], errors="coerce")
df_tfp["logWorkers"] = pd.to_numeric(df_tfp["logWorkers"], errors="coerce")
df_tfp["logPower"] = pd.to_numeric(df_tfp["logPower"], errors="coerce")

# Drop any new NaNs
df_tfp = df_tfp.dropna(subset=["logRev", "logWorkers", "logPower"])

print(f"TFP calculation dataset shape: {df_tfp.shape}")

# Run Cobb-Douglas regression: logRev ~ logWorkers + logPower (NO FIXED EFFECTS)
X = df_tfp[["logWorkers", "logPower"]]
X = sm.add_constant(X)
y = df_tfp["logRev"]

X = X.astype(float)
y = y.astype(float)

model = sm.OLS(y, X).fit()
df_tfp["TFP"] = model.resid

# Merge TFP into main dataframe
df = df.merge(df_tfp[["id", "TFP"]], on="id", how="left")



TFP calculation dataset shape: (15435, 65)


In [12]:
# === STEP 2: FIXED EFFECTS REGRESSIONS (Table 5 using PanelOLS with RegIndGroup clustering) ===
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

# Define outcomes
outcomes = ["logRevperWorker", "logPowerperWorker", "TFP"]

# Drop rows with missing values
df_fe = df.dropna(subset=outcomes + ["Form", "YEAR", "factory_id", "Province", "Industry"]).copy()

# Convert to numeric
for outcome in outcomes:
    df_fe[outcome] = pd.to_numeric(df_fe[outcome], errors='coerce')

df_fe = df_fe.dropna(subset=outcomes)

# Step 1: Create RegIndGroup for clustering
df_fe["RegIndGroup"] = df_fe["Province"].astype(str) + "_" + df_fe["Industry"].astype(str)

# Step 2: Set panel index
df_fe = df_fe.set_index(["factory_id", "YEAR"])

print(f"Fixed effects dataset shape: {df_fe.shape}")
print("\n=== Table 5 Fixed Effects Results (with PanelOLS and RegIndGroup clustering) ===")

# Step 3: Run PanelOLS regressions
for outcome in outcomes:
    exog = sm.add_constant(df_fe["Form"])
    model = PanelOLS(df_fe[outcome], exog, entity_effects=True)
    result = model.fit(cov_type="clustered", clusters=df_fe["RegIndGroup"])

    coef = result.params["Form"]
    se = result.std_errors["Form"]
    pval = result.pvalues["Form"]
    ci_low, ci_high = result.conf_int().loc["Form"]

    print(f"\nOutcome: {outcome}")
    print(f"  Coefficient: {coef:.3f}")
    print(f"  Std. Error:  {se:.3f}")
    print(f"  P-value:     {pval:.4f}")
    print(f"  95% CI:      [{ci_low:.3f}, {ci_high:.3f}]")


Fixed effects dataset shape: (15435, 64)

=== Table 5 Fixed Effects Results (with PanelOLS and RegIndGroup clustering) ===

Outcome: logRevperWorker
  Coefficient: 0.269
  Std. Error:  0.161
  P-value:     0.0954
  95% CI:      [-0.047, 0.585]

Outcome: logPowerperWorker
  Coefficient: 0.285
  Std. Error:  0.232
  P-value:     0.2199
  95% CI:      [-0.171, 0.741]

Outcome: TFP
  Coefficient: 0.086
  Std. Error:  0.179
  P-value:     0.6325
  95% CI:      [-0.266, 0.437]
